<h1>Dependencies</h1>

In [1]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

<h1>Embed Class Slides and Textbooks (only do this if embedding new documents)</h1>

In [ ]:
# Load documents
loader = DirectoryLoader('../docs', glob="**/*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(documents)

# Create embeddings and vector store
embeddings = OllamaEmbeddings(model="llama3.2", base_url="http://localhost:11434")
vectorstore = Chroma.from_documents(persist_directory='../chroma', documents=splits, embedding=embeddings)


<h1>Initialize Llama LLM locally</h1>

In [2]:
embeddings = OllamaEmbeddings(model="llama3.2")
vectorstore = Chroma(persist_directory='../chroma', embedding_function=embeddings)
# Initialize LLama 3.2
llm = Ollama(model="llama3.2", base_url="http://localhost:11434")



/tmp/ipykernel_96990/2116260297.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory='../chroma', embedding_function=embeddings)
/tmp/ipykernel_96990/2116260297.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2", base_url="http://localhost:11434")


<h1>Generate prompt with GUI using gradio</h1>

In [ ]:
import gradio as gr

def answer(message: str, history: list[str]):
    #Use llm to determine if question is relevant to network security
    relevant = 'yes' in llm(f"Question: Is {message} relevant to network security?").lower()
    
    #Find similar docs if relevant
    similar_docs = vectorstore.similarity_search(message, k=10) if relevant else []

    #Document the source
    source = "\n".join([f"Source: {document.metadata['source']} Page Number: {document.metadata['page']}" for document in similar_docs]) if relevant else "Source: Not relevant to network security."

    #Construct prompt using history from past requests - this will allows to take interactive quizzes
    prompt = "\n".join([f"User: {item[0]}\nBot: {item[1]}" for item in history]) + f"\nUser: {message}\nBot:"
    
    #Construct context if relevant
    context = "\n".join([doc.page_content for doc in similar_docs]) if relevant else []

    #Send constructed prompt to llm and add message, result to history
    response = llm(f"Context: {context}\nQuestion: {prompt}\nAnswer:")
    history.append([message, response])

    #Display the reponse as well as the source
    return (response + "\n\n" + source)


chatbot = gr.ChatInterface(
    fn=answer,
    title="Group 5 Quizbot",
    description="Enter the following prompt to be quizzed: Generate a multiple choice, true/false, or short answer network security question and provide feedback to my answer."
)

chatbot.launch(server_port=7860)


/home/nathan/GIT_branches/CS5342Project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/tmp/ipykernel_96990/2050464858.py:5: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant = 'yes' in llm(f"Question: Is {message} relevant to network security?").lower()
